In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-1.shtml'
# 得到页面的内容
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
html=requests.get(url,headers=headers,timeout=10)
content = html.text
# 通过content创建BeautifulSoup对象
soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')


/Users/kidwishzhu/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [3]:
#输出第一个 title 标签
print(soup.title)
#输出第一个 title 标签的标签名称
print(soup.title.name)
#输出第一个 title 标签的包含内容
print(soup.title.string)


<title>汽车质量投诉 - 车质网</title>
title
汽车质量投诉 - 车质网


In [6]:
#输出第一个 title 标签
print(soup.title)
#输出第一个 title 标签的标签名称
print(soup.title.name)
#输出第一个 title 标签的包含内容
print(soup.title.string)
#输出第一个 title 标签的父标签的标签名称
print(soup.title.parent.name)
 

#输出第一个p标签
print(soup.p)
 #输出第一个  p 标签的 class 属性内容
print(soup.p['class'])
 
#输出第一个  a 标签的  href 属性内容
print(soup.a['href'])
# soup的属性操作方法与字典一样，可以被添加,删除或修改. 
# 修改第一个 a 标签的href属性为 http://www.baidu.com/
soup.a['href'] = 'http://www.baidu.com/'
#给第一个 a 标签添加 name 属性
soup.a['name'] = u'百度'
print(soup.a)
#删除第一个 a 标签的 class 属性为
del soup.a['class']
print(soup.a)



<title>汽车质量投诉 - 车质网</title>
title
汽车质量投诉 - 车质网
head
<p class="lxwm">
<a href="http://www.12365auto.com/copyright.shtml" target="_blank">版权声明</a>|<a href="http://www.12365auto.com/about.shtml" target="_blank">关于我们</a>|<a href="http://www.12365auto.com/contact.shtml" target="_blank">联系我们</a>|<a href="http://www.12365auto.com/job.shtml" target="_blank">诚聘英才</a>|<a href="http://www.12365auto.com/links.shtml" target="_blank">友情链接</a>|<a href="http://www.12365auto.com/bug" target="_blank">意见反馈</a></p>
['lxwm']
http://www.12365auto.com
<a href="http://www.baidu.com/" name="百度">
<img src="/x_images/logo.jpg"/></a>
<a href="http://www.baidu.com/" name="百度">
<img src="/x_images/logo.jpg"/></a>


In [9]:
# 对汽车投诉信息进行数据预处理
import pandas as pd

# 分析type的内容，拆分成 年份, 发动机, 变速器（手动/自动），其他
def analyze(type):
	# 一般第一个为年份
	#year = type[0]
	year, engine, transmission, other = '', '', '', ''
	engine_list = ['1.2T', '1.4T', '1.4L', '1.4TSI', '1.5L', '1.5T', '1.5TD', '1.6L', '1.6T', '1.6THP', '1.8L', '1.8T', '1.8TD', '1.8TSI', '2.0L', '2.0T', '2.4L', '2.5L', '2.5T', '14T', '20T', '30T', '230TSI', '350T', '280TSI', '260T', '300T', '300TGI', '330TSI', '350THP', '350T', 'TSI280', '400TGI']
	for i in type:
		# 如果最后一个字为款，为年份
		if type.index(i)==0 and i[-1:] == '款':
			year = i[:-1]
			continue
		if i == '手动' or i == '自动':
			transmission = i
			continue
		if i in engine_list: 
			engine = i
			continue
		other = other + ' ' + i
	return year, engine, transmission, other


# 数据加载
result = pd.read_csv('car_complain.csv')
result['type_year'] = ''
result['type_engine'] = ''
result['type_transmission'] = ''
result['type_other'] = ''

# 分析type字段，拆分多个字段
for i, row in result.iterrows():
	year, engine, transmission, other = analyze(row['type'].split(' '))
	result.loc[i, 'type_year'] = year
	result.loc[i, 'type_engine'] = engine
	result.loc[i, 'type_transmission'] = transmission
	result.loc[i, 'type_other'] = other
	#print(year, engine, transmission, other)
# 删除列
result = result.drop(columns=['type'], axis=1)
result.to_csv('car_complain_data_clean.csv', index=False)
